In [6]:
from collections import defaultdict

In [68]:
data = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""

# data = """.....
# .S-7.
# .|.|.
# .L-J.
# ....."""

In [69]:
with open("input.txt") as f:
    data = f.read()

In [70]:

dirs = {
    'S': ((0,1), (1,0), (0,-1), (-1,0)), 
    '|': ((0,1), (0,-1)),
    '-': ((1,0), (-1,0)),
    '7': ((0,1), (-1,0)),
    'J': ((0,-1), (-1,0)),
    'L': ((0,-1), (1,0)),
    'F': ((0,1), (1,0)),
    '.': (),
}

grid = defaultdict(lambda: '.')

for y, line in enumerate(data.splitlines()):
    for x, c in enumerate(line):
        grid[(x, y)] = c
        if c == 'S':
            start = (x, y)
            


In [71]:

# Graphic representation of the grid
dirs_to_img = {
    'S': '\033[0;30;41m@\033[0;0m',
    '|': '║',
    '-': '═',
    '7': '╗',
    'J': '╝',
    'L': '╚',
    'F': '╔',
    '.': '\033[0;30;44m \033[0;0m',
    '*': '\033[0;30;41m \033[0;0m'
}

def print_grid(grid, inside_nodes={}, display_raw=False, crop=False):
    max_x = max(x for x, _ in grid.keys())
    max_y = max(y for _, y in grid.keys())
    assert max_x < 1000
    assert max_y < 1000
    for y in range(max_y + 1):
        if crop is not False and y not in range(crop[2], crop[3]):
            continue
        for x in range(max_x + 1):
            if crop is not False and x not in range(crop[0], crop[1]):
                continue
            if (x, y) in inside_nodes:
                print('\033[0;30;42m \033[0;0m', end='')
            elif display_raw:
                print(grid[(x, y)].replace('.', '\033[0;30;44m \033[0;0m'), end='')
            else:
                print(dirs_to_img[grid[(x, y)]], end='')
        print()
        
# print_grid(grid)

In [74]:
path = [start]

try:
    while True:
        x, y = path[-1]
        path.append(next((x+dx, y+dy) 
                        for dx,dy in dirs[grid[(x, y)]]
                        for dir in dirs[grid[(x+dx, y+dy)]]
                        if dir == (-dx, -dy) and (x+dx, y+dy) not in path[1:]))
except StopIteration:
    pass

print("Part 1: ", len(path[:-1])//2)

Part 1:  6823


In [32]:
import math
# signed angle between two vectors
def turn(dx1, dy1, dx2, dy2):
    angle = math.atan2(dx1*dy2 - dy1*dx2, dx1*dx2 + dy1*dy2)
    if angle < 0:
        return -1
    elif angle > 0:
        return 1
    else:
        return 0

In [77]:
# print([grid[p] for p in path])

diffs = [(x2-x1, y2-y1) for (x1, y1), (x2, y2) in zip(path, path[1:])]
turns = [turn(dx1, dy1, dx2, dy2) for (dx1, dy1), (dx2, dy2) in zip(diffs, diffs[1:])]
turns = [turn(*diffs[-1], *diffs[0]), *turns]

# print(diffs)
print(sum(turns))

4


In [78]:
# rotate vector (dx, dy) by 90 degrees CW
def rotate_cw(dx, dy):
    return dy, -dx

# rotate vector (dx, dy) by 90 degrees CCW
def rotate_ccw(dx, dy):
    return -dy, dx


In [79]:
inside_nodes = set()
clean_grid = {k: v if k in path else '.' for k, v in grid.items()}

rot = rotate_ccw if sum(turns) > 0 else rotate_cw

for (x,y), (dx, dy), (dx2, dy2) in zip(path[1:], diffs, [*diffs[1:], diffs[0]]):
    ddx, ddy = rot(dx, dy)
    inside_neigh = x+ddx, y+ddy
    if clean_grid.get(inside_neigh, False) == '.' and not inside_neigh in inside_nodes:
        inside_nodes.add(inside_neigh)

    # Dirty hack to catch angles:
    if dx2 != dx and dy2 != dy:
        ddx, ddy = rot(dx2, dy2)
        inside_neigh = x+ddx, y+ddy
        if clean_grid.get(inside_neigh, False) == '.' and not inside_neigh in inside_nodes:
            inside_nodes.add(inside_neigh)

print_grid(clean_grid, inside_nodes)


                                                                                                                                            
                                                                                                                                            
                                                                                                                                            
                                       ╔══╗                                                          ╔╗                                     
                                      ╔╝╔═╝ ╔╗                                                    ╔╗╔╝╚══╗                                  
                                     ╔╝╔╝╔╗╔╝║╔══╗  ╔╗                                            ║╚╝╔═╗╔╝                                  
                                     ╚╗║ ║║║╔╝╚═╗╚══╝╚╗                                 ╔╗        ╚═╗╚╗╚╝                                   
             

In [81]:

# expand inside nodes
to_visit = list(inside_nodes)
visited = set()

while len(to_visit) > 0:
    x, y = to_visit.pop()
    visited.add((x, y))
    for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)):
        neigh = x+dx, y+dy
        if (neigh not in visited) and (neigh not in inside_nodes) and (clean_grid[neigh] == '.'):
            inside_nodes.add(neigh)
            to_visit.append(neigh)

print_grid(clean_grid, inside_nodes)
print("Part 2:", len(inside_nodes))

                                                                                                                                            
                                                                                                                                            
                                                                                                                                            
                                       ╔══╗                                                          ╔╗                                     
                                      ╔╝╔═╝ ╔╗                                                    ╔╗╔╝╚══╗                                  
                                     ╔╝╔╝╔╗╔╝║╔══╗  ╔╗                                            ║╚╝╔═╗╔╝                                  
                                     ╚╗║ ║║║╔╝╚═╗╚══╝╚╗                                 ╔╗        ╚═╗╚╗╚╝                                   
             